In [52]:
import pandas as pd
import numpy as np
import nltk
import scipy.sparse as sps
import re
from scipy import sparse
from nltk import bigrams    
from unicodedata import normalize
from nltk.corpus import stopwords
from collections import Counter

In [53]:
leitura = pd.read_csv('date/estadao_noticias_eleicao.csv')

In [54]:
def text_clear(text):
    pattern = re.compile('[^a-zA-Z0-9 ]')
    text = normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')
    return pattern.sub(' ', text)

In [55]:
words = [text_clear(stopword) for stopword in stopwords.words('portuguese')]

In [56]:
conteudos = leitura.titulo + " " + leitura.subTitulo +  " " + leitura.conteudo
conteudos  = conteudos.fillna("")
conteudos = conteudos.apply(text_clear)
ids = leitura.idNoticia

In [57]:
noticias = conteudos.apply(nltk.word_tokenize)
freq_term = noticias.apply(Counter)

In [58]:
tokens_lists = conteudos.apply(lambda text: text.lower().split())
noticias_token = [token for tokens_list in tokens_lists for token in tokens_list if token not in words]

In [59]:
index = {}
for i in range(len(noticias)):
    id_noticia = ids[i]
    for palavra in noticias[i]:
        palavra = palavra.lower()
        if palavra not in index:
            index[palavra] = {}
        id_rec = index[palavra].get(id_noticia)
        
        if not id_rec:
            docs = index[palavra]
            docs[id_noticia] = freq_term[i][palavra]

In [70]:
def generator_docs_peso(termos):
    docs_peso = {}
    
    for i in range(len(termos)):
        termo = termos[i]
        docs = index[termo]
        for doc_id in docs:
            tf = docs[doc_id]
            
            if doc_id not in docs_peso:
                docs_peso[doc_id] = np.array([0 if j != i else tf for j in range(len(termos))])
            else:
                doc_vector = docs_peso[doc_id]
                doc_vector[i] = tf
    return docs_peso

In [73]:
def generator_query(phase):
    query = np.array([1 if index.get(termo) else 0 for termo in phase])
    return query

In [74]:
def seach_tf(phase):
    docs_tf = generator_docs_peso(phase)
    query = generator_query(phase)
    doc_rank = sorted(list(docs_tf.items()), key=lambda doc: np.dot(doc[1], query), reverse=True)[:5] 
    return [doc[0] for doc in doc_rank]

In [63]:
def co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    n = len(vocab)
   
    vocab_to_index = {word:i for i, word in enumerate(vocab)}
    
    bi_grams = list(bigrams(corpus))

    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))

    I=list()
    J=list()
    V=list()
    
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]

        I.append(vocab_to_index[previous])
        J.append(vocab_to_index[current])
        V.append(count)
        
    co_occurrence_matrix = sparse.coo_matrix((V,(I,J)), shape=(n,n))

    return co_occurrence_matrix, vocab_to_index

In [64]:
matrix, vocab =co_occurrence_matrix(noticias_token)

# Consult Frequencia

In [65]:
matrix_consult = matrix.tocsr()

In [66]:
def consult_frequency(w1, w2):
    return(consultable_matrix[vocab[w1],vocab[w2]])

In [67]:
def get_ocorrence(palavra):
    list_occurency = matrix_consult[vocab[palavra]].getrow(0).toarray()[0]
    indexs, frequency = zip(*sorted(enumerate(list_occurency), key=lambda x: x[1], reverse=True))
    return indexs[:3], frequency[:3]

In [75]:
termo = 'lula'
ocurrecy = get_ocorrence(termo)
expansao = [word for key in ocurrecy[0] for word in vocab.keys() if vocab[word] == key]
expansao.append(termo)
seach_tf(expansao)

[2871, 2890, 3954, 4235, 4750]


[2871, 2890, 3954, 4235, 4750]